In [1]:
lappend auto_path [file dirname [info script]]/lib
tcl::tm::path add [file dirname [info script]]/modules [file dirname [info script]]/lib/cintcode
package require util

In [2]:
set data [split [read-input day14] \n];

In [3]:
set ex1 [split {10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL} \n]
set ex2 [split {9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL} \n];

In [4]:
proc book data {
    set book {}
    foreach line $data {
        set parts [lassign [lreverse [lmap x [split [string map {", " @ " => " @ " "  @ } $line] @] {string trim $x}]] comp amount]
        dict set book $comp [list $amount $parts]
    }
    return $book
}


In [5]:
proc reqs {book comp reqamount stock} {

    lassign [dict get $book $comp] amount parts

    set extra 0
    set times [expr {$reqamount / $amount}]
    set extra [expr {$reqamount % $amount}]
    if {$extra != 0} {
        incr times
        dict incr stock $comp [expr {$amount - $extra}]
    }
    set reqs {}
    foreach {req amount} $parts {
        lappend reqs $req [expr {$amount * $times}]
    }
    return [list $reqs $stock]
}
reqs [book $data] FUEL 1 {}

In [6]:
proc produce {book parts stock} {
    set res {}
    foreach {part reqamount} $parts {
        if {$part eq "ORE"} {
            dict incr res $part $reqamount
            continue
        }
        if {[dict exists $stock $part]} {
        set instock [dict get $stock $part]
        if {$instock > $reqamount } {
            dict incr stock $part -$reqamount
            continue
        } else {
            set stock [dict remove $stock $part]
            incr reqamount -$instock
        }
        }
        lassign [reqs $book $part $reqamount $stock] reqs stock
        foreach {req amount} $reqs {

            dict incr res $req $amount
        } 
    }
    return [list $res $stock]
}


# Part 1

In [7]:
proc part1 {data} {
    set reqs {FUEL 1}
    set stock {}
    while {1} {
        lassign  [produce [book $data] $reqs $stock] reqs stock
        if {[llength $reqs] == 2 } break
    }
    return $reqs
}
part1 $data

ORE 485720

# Part 2

In [10]:
set book [book $data];


In [11]:
proc part2 {ores book} {
set upper $ores
set lower 0


while 1 {
    set try [expr {($upper-$lower)/2+$lower}]
    puts "Trying $try"
    set reqs [list FUEL $try]
    set stock {}
    while {1} {
        lassign  [produce $book $reqs $stock] reqs stock
        if {[llength $reqs] == 2 } break
    }
    set ores [lindex $reqs end]
    if { $ores > 1000000000000} {
        set upper $try
    } else {
        set lower $try
    }
    if {$upper - $lower == 1} {
        puts "Result $lower"
        break
    }
}
return $lower
}
part2 1000000000000 [book $data]

Trying 500000000000
Trying 250000000000
Trying 125000000000
Trying 62500000000
Trying 31250000000
Trying 15625000000
Trying 7812500000
Trying 3906250000
Trying 1953125000
Trying 976562500
Trying 488281250
Trying 244140625
Trying 122070312
Trying 61035156
Trying 30517578
Trying 15258789
Trying 7629394
Trying 3814697
Trying 5722045
Trying 4768371
Trying 4291534
Trying 4053115
Trying 3933906
Trying 3874301
Trying 3844499
Trying 3859400
Trying 3851949
Trying 3848224
Trying 3850086
Trying 3849155
Trying 3848689
Trying 3848922
Trying 3849038
Trying 3848980
Trying 3849009
Trying 3848994
Trying 3849001
Trying 3848997
Trying 3848999
Trying 3848998
Result 3848998


3848998